<a href="https://colab.research.google.com/github/CapMayank01/CapMayank01/blob/main/FakeNewDetrctor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords # the,for,of,it,with
from nltk.stem.porter import PorterStemmer #loved loving == love (root word)
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
news_df = pd.read_csv('/content/WELFake_Dataset.csv')

In [4]:
news_df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [5]:
news_df.shape

(72134, 4)

In [6]:
news_df.isna().sum()

,0
Unnamed: 0,0
title,558
text,39
label,0


In [7]:
news_df = news_df.fillna(' ')

In [9]:
news_df.isna().sum()

,0
Unnamed: 0,0
title,0
text,0
label,0


In [10]:
news_df['content'] = news_df['title']

In [11]:
news_df


,Unnamed: 0,title,text,label,content
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,LAW ENFORCEMENT ON HIGH ALERT Following Threat...
1,1,,Did they post their votes for Hillary already?,1,
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,"Bobby Jindal, raised Hindu, uses story of Chri..."
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,SATAN 2: Russia unvelis an image of its terrif...
...,...,...,...,...,...
72129,72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0,Russians steal research on Trump in hack of U....
72130,72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1,WATCH: Giuliani Demands That Democrats Apolog...
72131,72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0,Migrants Refuse To Leave Train At Refugee Camp...
72132,72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0,Trump tussle gives unpopular Mexican leader mu...


In [14]:
news_df['content']

,content
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...
1,
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...
3,"Bobby Jindal, raised Hindu, uses story of Chri..."
4,SATAN 2: Russia unvelis an image of its terrif...
...,...
72129,Russians steal research on Trump in hack of U....
72130,WATCH: Giuliani Demands That Democrats Apolog...
72131,Migrants Refuse To Leave Train At Refugee Camp...
72132,Trump tussle gives unpopular Mexican leader mu...


In [15]:
news_df['content'][71229]

'COLLEGE STUDENTS Who Wanted You To Pay For Their Education Make STUNNING Admissions About How They Spend Student Loan Money'

In [20]:
#stemming

ps = PorterStemmer()

def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [25]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [26]:
news_df['content'] = news_df['content'].apply(stemming)

In [27]:
news_df['content']

,content
0,law enforc high alert follow threat cop white ...
1,
2,unbeliev obama attorney gener say charlott rio...
3,bobbi jindal rais hindu use stori christian co...
4,satan russia unv imag terrifi new supernuk wes...
...,...
72129,russian steal research trump hack u democrat p...
72130,watch giuliani demand democrat apolog trump ra...
72131,migrant refus leav train refuge camp hungari
72132,trump tussl give unpopular mexican leader much...


In [29]:
X = news_df['content'].values
Y = news_df['label'].values

In [30]:
print(X)

['law enforc high alert follow threat cop white blacklivesmatt fyf terrorist video'
 ''
 'unbeliev obama attorney gener say charlott rioter peac protest home state north carolina video'
 ... 'migrant refus leav train refuge camp hungari'
 'trump tussl give unpopular mexican leader much need shot arm'
 'goldman sach endors hillari clinton presid']


In [36]:
vector = TfidfVectorizer()
vector.fit(X)
X = vector.transform(X)

In [37]:
print(X)


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 629110 stored elements and shape (72134, 19639)>
  Coords	Values
  (0, 407)	0.3190180925014663
  (0, 1802)	0.33473541566384035
  (0, 3679)	0.24871262252022117
  (0, 5509)	0.31820565801047196
  (0, 6425)	0.28932771754845743
  (0, 6730)	0.48553136502134386
  (0, 7887)	0.26746434949988324
  (0, 9699)	0.22829788917209384
  (0, 17260)	0.24871262252022117
  (0, 17363)	0.2542650376115143
  (0, 18648)	0.1297506867782943
  (0, 19106)	0.19134939529376566
  (2, 1049)	0.28404017886581956
  (2, 2673)	0.30809679188606154
  (2, 2919)	0.3639616996972358
  (2, 6880)	0.2652283770602196
  (2, 8020)	0.2692285294185893
  (2, 11864)	0.2231406266784195
  (2, 12011)	0.16878852994653004
  (2, 12744)	0.27904818164471595
  (2, 13591)	0.22687620695463123
  (2, 14591)	0.3580030298678158
  (2, 15094)	0.1609967301122813
  (2, 16446)	0.1999703023632961
  (2, 18034)	0.35962437110547785
  :	:
  (72130, 17778)	0.13227219506940732
  (72130, 18936)	0.2530249939

In [42]:
X_train , X_test , Y_train , Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=1)

In [40]:
X_train.shape0


(57707, 19639)

In [43]:
model = LogisticRegression()
model.fit(X_train,Y_train)

LogisticRegression()

In [45]:
train_y_pred = model.predict(X_train)
print(accuracy_score(train_y_pred,Y_train))

0.9202696379988563


In [46]:
test_y_pred = model.predict(X_test)
print(accuracy_score(test_y_pred,Y_test))

0.8988008594995495


In [81]:

input_text = 'Russian oil presented as an issue Jaishankar on Trump tariffs on India'
# Preprocess the input text
stemmed_input = stemming(input_text)
input_data = vector.transform([stemmed_input])
prediction = model.predict(input_data)
print(prediction)
if (prediction[0]==1):
  print('The news is fake')
else:
  print('The news is Real')

[0]
The news is Real


In [63]:
news_df['content'][100]

'one person shot portland anti trump protest cross bridg polic'

In [84]:
news_df['content'][1020]

'white hous staff reportedli went behind trump back huge issu trump reckless'

In [ ]:
#some latest news reports
#CBI books Anil Ambani's RCOM for ₹2,000-crore bank fraud, searches premises
#From Abbottabad to Trump: The unbroken chain in U.S.–Pakistan military relations
#Indians in Ireland: Building a home amid the hostility On July 19, 2025, an Indian man was assaulted in the suburb of Tallaght in Dublin, Ireland. Since then, there have been several attacks against Indians in the green isle, causing alarm in the immigrant community. With politicians there riding on anti-immigrant rhetoric, the situation has worsened. Priyanka Borpujari reports on how the transformation of Ireland as a major immigrant destination has brought many challenges

In [83]:

while True:
    input_text = input("Enter the news article text (or type 'exit' to quit): ")

    if input_text.lower() == 'exit':
        print("Thank you for using the fake news detector!")
        break

    if input_text:
        prediction_result = predict_fake_news(input_text)
        print(f"Prediction: {prediction_result}")
    else:
        print("Please enter some text to predict.")

    while True:
        verify_more = input("Do you want to verify more news? (y/n): ").lower()
        if verify_more in ['y', 'n']:
            break
        else:
            print("Invalid input. Please enter 'y' or 'n'.")

    if verify_more == 'n':
        print("Thank you for using the fake news detector!")
        break

Enter the news article text (or type 'exit' to quit): one person shot portland anti trump protest cross bridg polic
Prediction: The news is fake
Do you want to verify more news? (y/n): y
Enter the news article text (or type 'exit' to quit): From Abbottabad to Trump: The unbroken chain in U.S.–Pakistan military relations 
Prediction: The news is Real
Do you want to verify more news? (y/n): y
Enter the news article text (or type 'exit' to quit): Russian oil presented as an issue Jaishankar on Trump tariffs on India
Prediction: The news is Real
Do you want to verify more news? (y/n): n
Thank you for using the fake news detector!
